In [1]:
import os
import pandas as pd
import re
import numpy as np

In [2]:
source = r"..\html"
directories = [x[0] for x in os.walk(source)]

In [36]:
volume = 2

In [59]:
files = []
for directory in directories:
    for file in os.listdir(directory):
        if file.endswith(".html") and file[2] == "_" and file[5] != "-":
            files.append(file)

In [38]:
df = pd.DataFrame(data=files, columns=["filename"])
df["folder"] = df["filename"].str[:2]
df["volume"] = df["filename"].str[1]
df["item"] = df["filename"].str[3:5]
df["path"] = source + "\\" + df["folder"] + "\\" + df["filename"]
df = df[["path", "filename", "folder", "volume", "item"]]

In [57]:
df.head()

,path,filename,folder,volume,item
0,..\html\D1\D1_01.html,D1_01.html,D1,1,01
1,..\html\D1\D1_02.html,D1_02.html,D1,1,02
2,..\html\D1\D1_03.html,D1_03.html,D1,1,03
3,..\html\D1\D1_04.html,D1_04.html,D1,1,04
4,..\html\D1\D1_05.html,D1_05.html,D1,1,05


In [73]:
Dune = df[df["volume"] == str(volume)]["path"]

In [74]:
content = []
for file in Dune:
    with open(file, mode="r", encoding="UTF-8") as f:
        content.append(f.readlines())

In [41]:
class_dictionary = {
    "indent" : "text_content",
    "space-break1" : "text_content",
    "right" : "opening_caption",
    "space-break" : "opening_caption",
    "blockquote" : "opening_text",
    "noindent" : "opening_text",
    "linegroup" : "text_poem",
    "line" : "text_poem",
    "line1" : "text_poem",
    "linex" : "text_poem",
    "chapter-title" : "chapter_title",
    "normal" : "text_content",
    "normal-1" : "text_content",
    "normal1" : "text_content",
    "bodytext" : "text_content",
    "bodytext-left" : "text_caption",
    "bodytextb" : "text_content",
    "bodytextt" : "text_content",
    "extract" : "text_poem",
    "extractba" : "text_poem",
    "bodytext-leftz" : "opening_text",
    "bodytext-leftza" : "opening_text",
    "hanging1" : "opening_text",
    "hanging" : "opening_text",
    "extract-indent1" : "opening_text",
    "extract-indent" : "text_poem",
    "bodytext1" : "text_content",
    "extract-indent-a1" : "text_poem",
    "extract-indent-b1" : "text_poem",
    "extract2" : "text_content",
    "extract-3" : "text_poem",
    "" : ""}

In [42]:
def PartOfText(text):
    # Function to regognize what part of text the given string is    
    class_start = text.find("<p class=") + len("<p class=") + 1
    class_end = text.find(">") - 1
    
    if class_start < 10:
        text_part = ""
    else:
        class_type = text[class_start:class_end]
        text_part = class_dictionary[class_type]

    return text_part;

In [143]:
def ExtractText(text):
    # Function to extract just text from given string and remove all html elements
    text = text.strip()
    text = re.sub("<em>", "‘", str(text))         # Change "thoughts" from cursive to dialogue format start
    text = re.sub("</em>", "’", str(text))        # Change "thoughts" from cursive to dialogue format end
    text = re.sub("\r\n", "", text)               # Remove line feeds
    text = re.sub("\n", "", text)                 # Remove line feeds
    text = re.sub("</p>", "", text)               # Remove class endings end
    text = re.sub("<a id=.*>", "", text)          # Remove id="page." references
    text = re.sub("<.*blockquote.*>", "", text)
    text = re.sub("<.*head>", "", text)
    text = re.sub("<.*body>", "", text)
    text = re.sub("<.*title.*>", "", text)
    text = re.sub("<.*link.*>", "", text)
    text = re.sub("<.*meta.*>", "", text)
    text = re.sub("<.*xml.*>", "", text)
    text = re.sub("</html>", "", text)
    
    return text;

#### opening captions for book 2
```html
<h5 class="bodytext-center-1"><strong>—Tleilaxu Theorem (unproven)</strong></h5>
<h5 class="bodytext-center-1"><strong>—Muad’Dib: Lecture to the War College from The Stilgar Chronicle</strong></h5>
```

In [144]:
book = []
for k in range(0, len(content)):
    for i in range(0, len(content[k])):
        book.append(content[k][i])

In [187]:
len(book)

3237

In [157]:
book_classified = []
for i in range(1, len(book)):
    temp = ["",""]
    s = ExtractText(book[i])
    temp[0] = PartOfText(s)
    temp[1] = re.sub("<.* class=.*>", "", s)
    book_classified.append(temp)

In [ ]:
book_classified = []
for i in range(1, len(book)):
    temp = ["",""]
    s = ExtractText(book[i])
    temp[0] = PartOfText(s)
    temp[1] = re.sub("<p class=.*>", "", s)
    book_classified.append(temp)

In [165]:
book_condensed = ["",""]
for i in range(0, len(book_classified)):
    temp = ["",""]
    if book_classified[i][1] != "":
        temp = [book_classified[i][0], book_classified[i][1]]
        book_condensed.append(temp)

In [161]:
book_condensed[:5]

['',
 '',
 ['',
  '‘Such a rich store of myths enfolds Paul Muad’Dib, the Mentat Emperor, and his sister, Alia, it is difficult to see the real persons behind these veils. But there were, after all, a man born Paul Atreides and a woman born Alia. Their flesh was subject to space and time. And even though their oracular powers placed them beyond the usual limits of time and space, they came from human stock. They experienced real events which left real traces upon a real universe. To understand them, it must be seen that their catastrophe was the catastrophe of all mankind. This work is dedicated, then, not to Muad’Dib or his sister, but to their heirs—to all of us’.'],
 ['text_content',
  'Muad’Dib’s Imperial reign generated more historians than any other era in human history. Most of them argued a particular viewpoint, jealous and sectarian, but it says something about the peculiar impact of this man that he aroused such passions on so many diverse worlds.'],
 ['text_content',
  'Of c

In [166]:
del book_condensed[:3]

In [167]:
book_condensed[:5]

[['text_content',
  'Muad’Dib’s Imperial reign generated more historians than any other era in human history. Most of them argued a particular viewpoint, jealous and sectarian, but it says something about the peculiar impact of this man that he aroused such passions on so many diverse worlds.'],
 ['text_content',
  'Of course, he contained the ingredients of history, ideal and idealized. This man, born Paul Atreides in an ancient Great Family, received the deep ‘prana-bindu’ training from the Lady Jessica, his Bene Gesserit mother, and had through this a superb control over muscles and nerves. But more than that, he was a ‘mentat’, an intellect whose capacities surpassed those of the religiously proscribed mechanical computers used by the ancients.'],
 ['text_content',
  'Above all else, Muad’Dib was the ‘Kwisatz Haderach’ which the Sisterhood’s breeding program had sought across thousands of generations.'],
 ['text_content',
  'The Kwisatz Haderach, then, the one who could be “many pl

In [168]:
df_book_long = pd.DataFrame(book_condensed, columns=["part","content"])

df_book_long['content'] = df_book_long['content'] + ' '
df_book_long['chapter_count'] = df_book_long.groupby(df_book_long.part.eq('opening_caption').cumsum()).cumcount() + 1
df_book_long['line_count'] = df_book_long.groupby(df_book_long.part.ne('').cumsum()).cumcount() + 1
df_book_long['content'] = df_book_long.groupby(df_book_long.part.ne('').cumsum()).content.apply(lambda x : x.cumsum())
df_book_long['take'] = df_book_long['line_count'] == df_book_long.groupby(df_book_long.part.ne('').cumsum())['line_count'].transform('max')
df_book_long['text_part'] = df_book_long.groupby(df_book_long.part.ne('').cumsum()).part.apply(lambda x : x.cumsum())

In [169]:
df_book_long.head(10)

,part,content,chapter_count,line_count,take,text_part
0,text_content,Muad’Dib’s Imperial reign generated more histo...,1,1,True,text_content
1,text_content,"Of course, he contained the ingredients of his...",2,1,True,text_content
2,text_content,"Above all else, Muad’Dib was the ‘Kwisatz Hade...",3,1,True,text_content
3,text_content,"The Kwisatz Haderach, then, the one who could ...",4,1,True,text_content
4,text_content,"Think on the paradox, the failure implicit in ...",5,1,True,text_content
5,text_content,He did all these things and more.,6,1,True,text_content
6,text_content,Muad’Dib’s Qizarate missionaries carried their...,7,1,True,text_content
7,text_content,"He did this because capture of Arrakis, that p...",8,1,True,text_content
8,text_content,Here was another ingredient of ideal history: ...,9,1,True,text_content
9,text_content,"Without melange, Paul Muad’Dib could not proph...",10,1,True,text_content


In [170]:
df_book = df_book_long[df_book_long['take'] == True].reset_index()
df_book = df_book[['text_part', 'content', 'chapter_count']]

df_book['chapter_start'] = df_book['chapter_count'].apply(lambda x : x == 1)
df_book['chapter_num'] = df_book['chapter_start']
df_book['chapter_num'].replace({True:1, False:0}, inplace=True)

In [89]:
# DUNE 1
df_book['chapter'] = df_book.chapter_num.cumsum() - 1
df_book['chapter'] = df_book['chapter'].shift(-1).ffill()

df_book = df_book[['chapter', 'text_part', 'content']][1:]

In [171]:
# DUNE 2
df_book['chapter'] = df_book.chapter_num.cumsum() + 1
df_book['chapter'] = df_book['chapter'].shift(-1).ffill()

df_book = df_book[['chapter', 'text_part', 'content']]

In [172]:
df_book.iloc[16]

chapter                                                      2
text_part                                         text_content
content      Despite the murderous nature of the plot he ho...
Name: 16, dtype: object

In [179]:
print(df_book['content'].iloc[16])

Despite the murderous nature of the plot he hoped to devise, the thoughts of Scytale, the Tleilaxu Face Dancer, returned again and again to rueful compassion. 


In [180]:
df_book['opening_caption'] = df_book['content'].str[0] == "—"

df_book['text_content'] = df_book['opening_caption'].shift(-2).ffill() != False
df_book['opening_text'] = df_book['opening_caption'].shift(-1).ffill() != False
df_book['text_content'] = df_book['opening_caption'].shift(1).ffill() != False

In [181]:
text_content_arr = np.where(df_book['text_content'] == True)
for i in text_content_arr:
    df_book['text_part'].iloc[i] = 'text_content'

opening_caption_arr = np.where(df_book['opening_caption'] == True)
for i in opening_caption_arr:
    df_book['text_part'].iloc[i] = 'opening_caption'

opening_text_arr = np.where(df_book['opening_text'] == True)
for i in opening_text_arr:
    df_book['text_part'].iloc[i] = 'opening_text'

In [182]:
df_book['paragraph'] = df_book.groupby(df_book.opening_text.eq(True).cumsum()).cumcount() + 1
df_book.head()

,chapter,text_part,content,opening_caption,text_content,opening_text,paragraph
0,2.0,text_content,Muad’Dib’s Imperial reign generated more histo...,False,True,False,1
1,2.0,text_content,"Of course, he contained the ingredients of his...",False,False,False,2
2,2.0,text_content,"Above all else, Muad’Dib was the ‘Kwisatz Hade...",False,False,False,3
3,2.0,text_content,"The Kwisatz Haderach, then, the one who could ...",False,False,False,4
4,2.0,text_content,"Think on the paradox, the failure implicit in ...",False,False,False,5


In [183]:
df_book.to_csv(f'output/D{volume}.csv',
               index=False,
               columns=['chapter', 'paragraph', 'text_part', 'content'])